In [53]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import requests, time
from selenium import webdriver
from pymongo import MongoClient


In [54]:
#set up splinter browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless = False)

In [55]:
url = "https://www.macys.com/?cm_sp=navigation-_-top_nav-_-macys_icon"
browser.visit(url)

html = browser.html
soup = bs(html, "html.parser")


In [56]:
db_client = MongoClient('localhost',27017)
db = db_client['scrape_data'] 
collection = db['Data_Collection']


In [57]:
response = webdriver.Chrome() 
response.get(url)
response.maximize_window()

In [58]:

pop_up_close_button = response.find_elements_by_xpath('//div[@id="tinybox"]//a[@id="closeButton"]')
if pop_up_close_button:
    try:
        pop_up_close_button[0].click()
        sleep(1)
    except:
        pass

In [59]:
home_department_link = response.find_elements_by_xpath('//div[@id="mainNavigation"]/ul/li/a[contains(text(),"HOME")]')
if home_department_link:
    home_department_link[0].click()
    sleep(5)

In [60]:
home_decor_link = response.find_elements_by_xpath('//div[@id="localNavigationContainer"]//ul[@id="firstNavSubCat"]/li//a[contains(text(),"Home Decor")]')
if home_decor_link:
    home_decor_link[0].click()
    sleep(5)

In [61]:
#going to each pages using the next page button and collecting all the data
next_page = response.find_elements_by_xpath('//div[@id="filters"]//a[@class="arrowRight arrowButton paginationSpacer"]')
while next_page:
#converting the source code into an object for effectively parsing all the html content
	parser = etree.HTML(response.page_source)

	#collecting all the products into a list and going through each one to collect name and price
	all_products_listed =  parser.xpath('//div[@id="macysGlobalLayout"]//ul[@id="thumbnails"]/li[@class="productThumbnail borderless"]')
	for product in all_products_listed:
		name = product.xpath('.//div[@class="shortDescription"]//a[@class="productThumbnailLink "]/text()')
		price = product.xpath('.//div[@class="prices"]//span[@class="first-range "]/text()')
		name = ''.join(name).strip()
		price = ''.join(price).strip().replace('$','')
		scrapeDate = datetime.today().strftime('%Y-%m-%d')
		data = {"Product_Name":name,"Price":price,"ScrapeDate":scrapeDate}
		if name:
			#inserting data into the mongodb collections, one at a time
			insert_data = collection.insert_one(data).inserted_id
		else:
			print("data is not a valid one")
	#checking if there is a next page exist or not. if there  is one then going to that page by clicking the next button		
	next_page = response.find_elements_by_xpath('//div[@id="filters"]//a[@class="arrowRight arrowButton paginationSpacer"]')
	if next_page:
		next_page[0].click()
		sleep(5)

In [62]:
db_client.close()
#clocing the browser window
response.close()



In [65]:
print(db_client)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)
